In [1]:
import cv2
import socket
import time
import numpy as np
from matplotlib import pyplot as plt
import logging
import math
import threading

_SHOW_IMAGE = False

def show_image(title, frame, show=_SHOW_IMAGE):
    if show:
        plt.imshow(frame, cmap = None, interpolation = None)
        plt.show()
    else:
        cv2.imshow(title, frame)

def make_points(frame, line):
    height, width, _ = frame.shape
    slope, intercept = line
    y1 = height  # bottom of the frame
    y2 = int(y1 * 1 / 2)  # make points from middle of the frame down

    # bound the coordinates within the frame
    x1 = max(-width, min(2 * width, int((y1 - intercept) / slope)))
    x2 = max(-width, min(2 * width, int((y2 - intercept) / slope)))
    return [[x1, y1, x2, y2]]

def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

def length_of_line_segment(line):
    x1, y1, x2, y2 = line
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def display_lines(frame, lines, line_color=(0, 255, 0), line_width=10):
    line_image = np.zeros_like(frame)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image, (x1, y1), (x2, y2), line_color, line_width)
    line_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    return line_image

def display_heading_line(frame, steering_angle, line_color=(0, 0, 255), line_width=5, ):
    heading_image = np.zeros_like(frame)
    height, width, _ = frame.shape

    # figure out the heading line from steering angle
    # heading line (x1,y1) is always center bottom of the screen
    # (x2, y2) requires a bit of trigonometry

    # Note: the steering angle of:
    # 0-89 degree: turn left
    # 90 degree: going straight
    # 91-180 degree: turn right 
    steering_angle_radian = steering_angle / 180.0 * math.pi
    x1 = int(width / 2)
    y1 = height
    x2 = int(x1 - height / 2 / math.tan(steering_angle_radian))
    y2 = int(height / 2)

    cv2.line(heading_image, (x1, y1), (x2, y2), line_color, line_width)
    heading_image = cv2.addWeighted(frame, 0.8, heading_image, 1, 1)

    return heading_image

def compute_steering_angle(frame, lane_lines):
    """ Find the steering angle based on lane line coordinate
        We assume that camera is calibrated to point to dead center
    """
    if len(lane_lines) == 0:
        logging.info('No lane lines detected, do nothing')
        return -90

    height, width, _ = frame.shape
    if len(lane_lines) == 1:
        logging.debug('Only detected one lane line, just follow it. %s' % lane_lines[0])
        x1, _, x2, _ = lane_lines[0][0]
        x_offset = x2 - x1
    else:
        _, _, left_x2, _ = lane_lines[0][0]
        _, _, right_x2, _ = lane_lines[1][0]
        camera_mid_offset_percent = 0.02 # 0.0 means car pointing to center, -0.03: car is centered to left, +0.03 means car pointing to right
        mid = int(width / 2 * (1 + camera_mid_offset_percent))
        x_offset = (left_x2 + right_x2) / 2 - mid

    # find the steering angle, which is angle between navigation direction to end of center line
    y_offset = int(height / 2)

    angle_to_mid_radian = math.atan(x_offset / y_offset)  # angle (in radian) to center vertical line
    angle_to_mid_deg = int(angle_to_mid_radian * 180.0 / math.pi)  # angle (in degrees) to center vertical line
    steering_angle = angle_to_mid_deg + 90  # this is the steering angle needed by picar front wheel

    logging.debug('new steering angle: %s' % steering_angle)
    return steering_angle


def stabilize_steering_angle(curr_steering_angle, new_steering_angle, num_of_lane_lines, max_angle_deviation_two_lines=5, max_angle_deviation_one_lane=1):
    """
    Using last steering angle to stabilize the steering angle
    This can be improved to use last N angles, etc
    if new angle is too different from current angle, only turn by max_angle_deviation degrees
    """
    if num_of_lane_lines == 2 :
        # if both lane lines detected, then we can deviate more
        max_angle_deviation = max_angle_deviation_two_lines
    else :
        # if only one lane detected, don't deviate too much
        max_angle_deviation = max_angle_deviation_one_lane
    
    angle_deviation = new_steering_angle - curr_steering_angle
    if abs(angle_deviation) > max_angle_deviation:
        stabilized_steering_angle = int(curr_steering_angle
                                        + max_angle_deviation * angle_deviation / abs(angle_deviation))
    else:
        stabilized_steering_angle = new_steering_angle
    logging.info('Proposed angle: %s, stabilized angle: %s' % (new_steering_angle, stabilized_steering_angle))
    return stabilized_steering_angle

def detect_lane(frame):
    
    #frame = cv2.GaussianBlur(frame,(5,5),0)

    edges = detect_edges(frame)
    show_image('edges', edges)

    cropped_edges = region_of_interest(edges)
    show_image('edges cropped', cropped_edges)

    line_segments = detect_line_segments(cropped_edges)
    line_segment_image = display_lines(frame, line_segments)
    show_image("line segments", line_segment_image)

    lane_lines = average_slope_intercept(frame, line_segments)
    lane_lines_image = display_lines(frame, lane_lines)
    show_image("lane lines", lane_lines_image)

    return lane_lines, lane_lines_image, line_segment_image


def detect_edges(frame):
    # filter for red lane lines
    img_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    show_image("bgr", frame)
    img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    show_image("hsv", img_hsv)
    
#     # lower mask (0-10)
#     lower_red = np.array([0,120,45])
#     upper_red = np.array([10,255,255]) #10
#     mask0 = cv2.inRange(img_hsv, lower_red, upper_red)

#     # upper mask (170-180)
#     lower_red = np.array([168,100,70]) #170,40,40
#     upper_red = np.array([180,255,255])
#     mask1 = cv2.inRange(img_hsv, lower_red, upper_red)

#     # join my masks
#     mask = mask0+mask1

#     # set my output img to zero everywhere except my mask
#     output_img = frame.copy()
#     output_img[np.where(mask==0)] = 0

#     # or your HSV image, which I *believe* is what you want
#     output_hsv = img_hsv.copy()
#     output_hsv[np.where(mask==0)] = 0
    
    #     lower_blue = np.array([30, 40, 0])
    #     upper_blue = np.array([150, 255, 255])
    #     mask = cv2.inRange(hsv, lower_blue, upper_blue)
    #     show_image("blue mask", mask)
    
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 40]) #,,30
    mask = cv2.inRange(img_hsv, lower_black, upper_black)
    show_image("black mask", mask)
    
    # set my output img to zero everywhere except my mask
    output_img = frame.copy()
    output_img[np.where(mask==0)] = 0

    # or your HSV image, which I *believe* is what you want
    output_hsv = img_hsv.copy()
    output_hsv[np.where(mask==0)] = 0    
    
    

    # detect edges
    edges = auto_canny(output_hsv,0.33)
    #edges = cv2.Canny(output_hsv,100, 400)

    return edges


def region_of_interest(canny):
    height, width = canny.shape
    mask = np.zeros_like(canny)

    # only focus bottom half of the screen

    polygon = np.array([[
        (0, height * 1 / 2),
        (width, height * 1 / 2),
        (width, height),
        (0, height),
    ]], np.int32)

    cv2.fillPoly(mask, polygon, 255)
    show_image("mask", mask)
    masked_image = cv2.bitwise_and(canny, mask)
    return masked_image


def detect_line_segments(cropped_edges):
    # tuning min_threshold, minLineLength, maxLineGap is a trial and error process by hand
    rho = 1  # precision in pixel, i.e. 1 pixel
    angle = np.pi / 180  # degree in radian, i.e. 1 degree
    min_threshold = 10  # minimal of votes
    line_segments = cv2.HoughLinesP(cropped_edges, rho, angle, min_threshold, np.array([]), minLineLength=8,
                                    maxLineGap=4)

    if line_segments is not None:
        for line_segment in line_segments:
            logging.debug('detected line_segment:')
            logging.debug("%s of length %s" % (line_segment, length_of_line_segment(line_segment[0])))

    return line_segments


def average_slope_intercept(frame, line_segments):
    """
    This function combines line segments into one or two lane lines
    If all line slopes are < 0: then we only have detected left lane
    If all line slopes are > 0: then we only have detected right lane
    """
    lane_lines = []
    if line_segments is None:
        logging.info('No line_segment segments detected')
        return lane_lines

    height, width, _ = frame.shape
    left_fit = []
    right_fit = []

    boundary = 0.5
    left_region_boundary = width * (1 - boundary)  # left lane line segment should be on left 2/3 of the screen
    right_region_boundary = width * boundary # right lane line segment should be on left 2/3 of the screen

    for line_segment in line_segments:
        for x1, y1, x2, y2 in line_segment:
            if x1 == x2:
                logging.info('skipping vertical line segment (slope=inf): %s' % line_segment)
                continue
            fit = np.polyfit((x1, x2), (y1, y2), 1)
            slope = fit[0]
            intercept = fit[1]
            if slope < 0:
                if x1 < left_region_boundary and x2 < left_region_boundary:
                    left_fit.append((slope, intercept))
            else:
                if x1 > right_region_boundary and x2 > right_region_boundary:
                    right_fit.append((slope, intercept))

    left_fit_average = np.average(left_fit, axis=0)
    if len(left_fit) > 0:
        lane_lines.append(make_points(frame, left_fit_average))

    right_fit_average = np.average(right_fit, axis=0)
    if len(right_fit) > 0:
        lane_lines.append(make_points(frame, right_fit_average))

    #print('lane lines: %s' % lane_lines)  # [[[316, 720, 484, 432]], [[1009, 720, 718, 432]]]

    return lane_lines

# l, lane_lines_image,lg = detect_lane(cv2.imread("lane1.jpeg"))
# plt.imshow(lane_lines_image, cmap = None, interpolation = None)
# plt.show()

# l, lane_lines_image,lg = detect_lane(cv2.imread("lane2.jpeg"))
# plt.imshow(lane_lines_image, cmap = None, interpolation = None)
# plt.show()

##### EXAMPPLES:
# lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("lane1.jpeg"))
# print(lane_lines)

# plt.imshow(line_segments_image, cmap = None, interpolation = None)
# plt.show()
    
# new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

# #curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
# curr_steering_angle = new_steering_angle
# final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
# print(curr_steering_angle)

# plt.imshow(final_frame, cmap = None, interpolation = None)
# plt.show()

# #------------
# lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("lane2.jpeg"))
# print(lane_lines)

# plt.imshow(line_segments_image, cmap = None, interpolation = None)
# plt.show()
    
# new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

# #curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
# curr_steering_angle = new_steering_angle
# final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
# print(curr_steering_angle)

# plt.imshow(final_frame, cmap = None, interpolation = None)
# plt.show()

# #-------------
# lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("toTurnLane.png"))
# print(lane_lines)

# plt.imshow(line_segments_image, cmap = None, interpolation = None)
# plt.show()
    
# new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

# #curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
# curr_steering_angle = new_steering_angle
# final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
# print(curr_steering_angle)

# plt.imshow(final_frame, cmap = None, interpolation = None)
# plt.show()

# #-------------
# lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("hole.jpeg"))
# print(lane_lines)

# plt.imshow(line_segments_image, cmap = None, interpolation = None)
# plt.show()
    
# new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

# #curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
# curr_steering_angle = new_steering_angle
# final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
# print(curr_steering_angle)

# plt.imshow(final_frame, cmap = None, interpolation = None)
# plt.show()

#------------- IN5
lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("in51.jpeg")) #red
print(lane_lines)

plt.imshow(line_segments_image, cmap = None, interpolation = None)
plt.show()
    
new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

#curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
curr_steering_angle = new_steering_angle
final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
print(curr_steering_angle)

plt.imshow(final_frame, cmap = None, interpolation = None)
plt.show()

#-------------

lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("in52.jpeg")) #red
print(lane_lines)

plt.imshow(line_segments_image, cmap = None, interpolation = None)
plt.show()
    
new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

#curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
curr_steering_angle = new_steering_angle
final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
print(curr_steering_angle)

plt.imshow(final_frame, cmap = None, interpolation = None)
plt.show()

#--------------

lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("in53.jpeg")) #red
print(lane_lines)

plt.imshow(line_segments_image, cmap = None, interpolation = None)
plt.show()
    
new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

#curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
curr_steering_angle = new_steering_angle
final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
print(curr_steering_angle)

plt.imshow(final_frame, cmap = None, interpolation = None)
plt.show()

#---------------
lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("in54.jpeg")) #red
print(lane_lines)

plt.imshow(line_segments_image, cmap = None, interpolation = None)
plt.show()
    
new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

#curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
curr_steering_angle = new_steering_angle
final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
print(curr_steering_angle)

plt.imshow(final_frame, cmap = None, interpolation = None)
plt.show()

#-----------------

lane_lines, lane_lines_image, line_segments_image = detect_lane(cv2.imread("in55.jpeg")) #red
print(lane_lines)

plt.imshow(line_segments_image, cmap = None, interpolation = None)
plt.show()
    
new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

#curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
curr_steering_angle = new_steering_angle
final_frame = display_heading_line(lane_lines_image, curr_steering_angle)
print(curr_steering_angle)

plt.imshow(final_frame, cmap = None, interpolation = None)
plt.show()

[[[11, 480, 140, 240]], [[630, 480, 454, 240]]]


<Figure size 640x480 with 1 Axes>

84


<Figure size 640x480 with 1 Axes>

[[[143, 480, 375, 240]]]


...

<Figure size 640x480 with 1 Axes>

[[[72, 480, 265, 240]], [[590, 480, 552, 240]]]


<Figure size 640x480 with 1 Axes>

108


<Figure size 640x480 with 1 Axes>

In [2]:
import socket
import time
import array

import cv2
import numpy as np


robot_ip_address = "192.168.1.1"  # Change to applicable
robot_port       = 2001       # Change to applicable
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

flag = 0

def connect():
    s.connect((robot_ip_address, robot_port)) #connect actually
    s.send(bytes([115,60,0,0]))

def forward():
    flag = 1
    s.send(bytes([115,100,0,0]))
    s.send(bytes([1])) #FORWARD

    time.sleep(0.3)

    s.send(bytes([5])) #STOP
    
    time.sleep(2)
    flag = 0
    
    
    
def left(delay=0.2):
    flag = 1
    
    s.send(bytes([115,60,0,0]))
    s.send(bytes([1])) #FORWARD

    time.sleep(0.1)

    speed=0
    left=0
    right=132
    s.send(bytes([115,speed,left,right]))
    s.send(bytes([1])) #FORWARD
    time.sleep(delay)

    s.send(bytes([115,60,0,0]))
    s.send(bytes([1])) #FORWARD

    time.sleep(0.1)
    s.send(bytes([5])) #STOP
    
    time.sleep(2)
    flag = 0

def right(delay=0.2):
    flag = 1
    s.send(bytes([115,60,0,0]))
    s.send(bytes([1])) #FORWARD

    time.sleep(0.1)

    speed=0
    right=0
    left=132
    s.send(bytes([115,speed,left,right]))
    s.send(bytes([1])) #FORWARD
    time.sleep(delay)

    s.send(bytes([115,60,0,0]))
    s.send(bytes([1])) #FORWARD

    time.sleep(0.1)
    s.send(bytes([5])) #STOP
    
    time.sleep(2)
    flag = 0

def stop():
    flag = 1
    s.send(bytes([5]))
    flag = 0
connect()    

In [ ]:
#cap = cv2.VideoCapture("lanes(Converted).mov")



def readCamera():
    cap = cv2.VideoCapture('http://192.168.1.1:8080/?action=stream')
    cap.set(cv2.CAP_PROP_BUFFERSIZE,0)
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    ret,frame = cap.read()
    cap.release()
    if (ret == True):
        return frame

#cap = cv2.VideoCapture("http://192.168.1.1:8080/?action=stream")
#cap.set(cv2.CAP_PROP_BUFFERSIZE,0)

# if (cap.isOpened()== False): 
#     print("Error opening video stream or file")
ret = True
curr_steering_angle=90
while (ret):
    if (flag==0):
        #ret, frame = cap.read() 
        #start_time = time.time()
        
        frame = readCamera()
        lane_lines, lane_lines_image,lg = detect_lane(frame)

        new_steering_angle = compute_steering_angle(lane_lines_image, lane_lines)

        final_frame = display_heading_line(frame, curr_steering_angle)
        print("Steering Angle: ",curr_steering_angle)

        #cv2.waitKey(5000)
        #time.sleep(5000)

        #cv2.putText(final_frame, str(curr_steering_angle), (300, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), lineType=cv2.LINE_AA) 

        if (new_steering_angle == -90):
            cv2.putText(final_frame, str("STOP"+str(curr_steering_angle)), (300, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), lineType=cv2.LINE_AA) 
            print("STOPPED",str(curr_steering_angle))
            cv2.imshow('image',final_frame)
            
            #stop()
            x = threading.Thread(target=stop)
            x.start()
            x.join()
        else:
            curr_steering_angle = stabilize_steering_angle(curr_steering_angle, new_steering_angle, len(lane_lines))
            print(curr_steering_angle)
            if (curr_steering_angle > 80 and curr_steering_angle < 100):
                print("forward")
                cv2.putText(final_frame, str('forward at '+str(curr_steering_angle)), (300, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), lineType=cv2.LINE_AA) 
                cv2.imshow('image',final_frame)
                
                #forward()
                x = threading.Thread(target=forward)
                x.start()
                x.join()
            elif (curr_steering_angle < 90 and curr_steering_angle > -1):
                print("left at",(90-curr_steering_angle))
                cv2.putText(final_frame, str("left at"+str(90-curr_steering_angle)), (300, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), lineType=cv2.LINE_AA) 
                cv2.imshow('image',final_frame)
                
                #left((90-curr_steering_angle)/90)
                leftAngle = (90-curr_steering_angle)/90
                x = threading.Thread(target=left, args=(leftAngle,))
                x.start()
                x.join()
            elif (curr_steering_angle > 90):   
                print("right at",(curr_steering_angle-90))
                cv2.putText(final_frame, str("right at"+str(curr_steering_angle-90)), (300, 400), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), lineType=cv2.LINE_AA) 
                cv2.imshow('image',final_frame)
                
                #right((curr_steering_angle-90)/90)
                rightAngle = (curr_steering_angle-90)/90
                x = threading.Thread(target=right, args=(rightAngle,))
                x.start()
                x.join()
        #print("--- %s seconds ---" % (time.time() - start_time))
        #cv2.imshow('image',final_frame)
        #time.sleep(1)

        cv2.waitKey(30)

#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             cv2.destroyAllWindows()
#             cap.release()
#             break
        
        #go left 60 80-100 straight
        #go right 130

Steering Angle:  90
95
forward
Steering Angle:  95
STOPPED 95
Steering Angle:  95
90
forward
Steering Angle:  90
95
forward
Steering Angle:  95
98
forward
Steering Angle:  98
93
forward
Steering Angle:  93
88
forward
Steering Angle:  88
93
forward
Steering Angle:  93
STOPPED 93
Steering Angle:  93
88
forward
Steering Angle:  88
STOPPED 88
Steering Angle:  88
83
forward
Steering Angle:  83
88
forward
Steering Angle:  88
93
forward
Steering Angle:  93
88
forward
Steering Angle:  88
91
forward
Steering Angle:  91
STOPPED 91
Steering Angle:  91
90
forward
Steering Angle:  90
93
forward
Steering Angle:  93
88
forward
Steering Angle:  88
84
forward
Steering Angle:  84
79
left at 11
Steering Angle:  79
80
left at 10
Steering Angle:  80
80
left at 10
Steering Angle:  80
85
forward
Steering Angle:  85
88
forward
Steering Angle:  88
93
forward
Steering Angle:  93
98
forward
Steering Angle:  98
97
forward
Steering Angle:  97
92
forward
Steering Angle:  92
97
forward
Steering Angle:  97
96
forward

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import os
import tensorflow as tf
from matplotlib import pyplot as plt
from PIL import Image
import glob as glob

%matplotlib inline
import sys
sys.path.append('/Users/eisaadil/.local/lib/python3.6/site-packages/tensorflow/models/research/object_detection') # ~/tensorflow/models/research/object_detection
from utils import label_map_util
from utils import visualization_utils as vis_util

import matplotlib.pyplot as plt

import cv2
import socket
import time

MODEL_NAME = 'ssd'

# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('scripts', 'label_map.pbtxt')

NUM_CLASSES = 78

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def left(speed):
    s.send(bytes([115,speed,0,0]))
    s.send(bytes([5])) #STOP
    time.sleep(0.7)
    s.send(bytes([0])) #LEFT
    print("Turn Left")

def right(speed):
    s.send(bytes([115,speed,0,0]))
    s.send(bytes([5])) #STOP
    time.sleep(0.7)
    s.send(bytes([2])) #LEFT
    time.sleep(0.7)
    forward(speed)
    print("Turn Right")
    
def stop():
    print("Stop")
    s.send(bytes([5]))
    
def sixty(speed):
    print("60km/h")
    s.send(bytes([115,speed,0,0]))
    
def forty(speed):
    print("40km/h")
    s.send(bytes([115,speed,0,0]))    
    
def forward(speed):
    s.send(bytes([115,speed,0,0]))
    s.send(bytes([1]))
    print("Go Forward")  

In [ ]:
cap = cv2.VideoCapture("http://192.168.1.1:8080/?action=stream")
IMAGE_SIZE = (20, 20)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        if (cap.isOpened()== False): 
              print("Error opening video stream or file")
        forward(60)     
        ret = True
        while (ret):
            start_time = time.time()
            ret, image_np = cap.read() 

            # Image Shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis = 0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections], feed_dict = {
                image_tensor: image_np_expanded
              })
            
            if scores[0][0]>0.5:
                if (classes[0][0]==15): #LEFT
                    left(150)
                elif (classes[0][0]==40): #RIGHT
                    right(150)
                elif (classes[0][0]==76): #STOP
                    stop()
                elif (classes[0][0]==16): #60 SPEED
                    sixty(100)
                elif (classes[0][0]==67): #40 SPEED (DEFAULT) 
                    forty(60)
                elif (classes[0][0]==36):   
                    forward(60)  
            
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates = True,
              line_thickness = 8)
            
#             plt.figure(figsize = IMAGE_SIZE)
#             plt.imshow(image_np)
            cv2.imshow('image',cv2.resize(image_np,(1280,960)))
            print("--- %s seconds ---" % (time.time() - start_time))
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                cap.release()
                break

In [ ]:
s.close()